In [ ]:
from pymongo import MongoClient

In [ ]:
from pymongo import MongoClient
 
def get_active_workflows(mongo_uri: str, db_name: str):
    """
    Fetch all active workflows from TableWorkFlow collection.
   
    :param mongo_uri: MongoDB connection URI
    :param db_name: Database name
    :return: List of active workflow documents
    """
    client = MongoClient(mongo_uri)
    db = client[db_name]
 
    # Fetch documents where IsActive is True
    workflows = list(db.TableWorkFlow.find({"IsActive": True}))
 
    client.close()
    return workflows
 
 
# ---------------- TEST ----------------
if __name__ == "__main__":
    mongo_uri = "mongodb://localhost:27017"  # your MongoDB URI
    db_name = "VibeOP_Dev"                   # your database name
 
    active_workflows = get_active_workflows(mongo_uri, db_name)
 
    print(f"Found {len(active_workflows)} active workflows.\n")
    for wf in active_workflows:
        print("Workflow ID:", wf.get("TableWorkflowId"))
        print("Source Table:", wf.get("SourceTableName"))
        print("Target Table:", wf.get("TargetTableName"))
        print("Source Columns:", [c["SourceColumn"] for c in wf.get("SourceColumns", [])])
        print("Target Columns:", [c["TargetColumn"] for c in wf.get("TargetColumns", [])])
        print("Source Connection ID:",(mongo_uri ,"DatabaseConnections",wf.get("SourceConnectionId")))
        print("Target Connection ID:", wf.get("TargetConnectionId"))
        print("-" * 50)
 
 
def get_connection_string(mongo_uri: str, db_name: str, connection_id: str):
    """
    Fetch idleTimeoutSeconds (encrypted connection string) for a given Connection ID
    from the DatabaseConnections collection.
   
    :param mongo_uri: MongoDB connection URI
    :param db_name: Database name
    :param connection_id: Connection ID (_id in DatabaseConnections)
    :return: idleTimeoutSeconds string or None if not found
    """
    client = MongoClient(mongo_uri)
    db = client[db_name]
 
    # Ensure connection_id is ObjectId
    try:
        conn_obj_id = ObjectId(connection_id)
    except Exception as e:
        print(f"Invalid connection ID: {connection_id}. Error: {e}")
        return None
 
    conn_doc = db.DatabaseConnections.find_one({"_id": conn_obj_id})
    client.close()
 
    if conn_doc:
        return conn_doc.get("idleTimeoutSeconds")
    else:
        return None
 
 

In [ ]:
def get_connection_string(mongo_uri: str, db_name: str, connection_id: str):
    client = MongoClient(mongo_uri)
    db = client[db_name]

    try:
        conn_obj_id = ObjectId(connection_id)
    except Exception as e:
        print(f"Invalid connection ID: {connection_id}. Error: {e}")
        return None

    conn_doc = db.DatabaseConnections.find_one({"_id": conn_obj_id})
    client.close()

    if conn_doc:
        return conn_doc.get("idleTimeoutSeconds")  # encrypted
    return None




def decrypt_connection_string(encrypted_value: str) -> str:
    """
    Decrypt encrypted connection string
    """
    if not encrypted_value:
        return None

    # Example only – replace with your real decryption logic
    decrypted_value = encrypted_value[::-1]  # dummy logic
    return decrypted_value


In [ ]:
active_workflows = get_active_workflows(mongo_uri, db_name)

for wf in active_workflows:

    # --- SOURCE ---
    source_conn_id = wf.get("SourceConnectionId")
    encrypted_source_conn = get_connection_string(
        mongo_uri, db_name, source_conn_id
    )
    source_conn_string = decrypt_connection_string(encrypted_source_conn)

    # --- TARGET ---
    target_conn_id = wf.get("TargetConnectionId")
    encrypted_target_conn = get_connection_string(
        mongo_uri, db_name, target_conn_id
    )
    target_conn_string = decrypt_connection_string(encrypted_target_conn)

    print("Workflow:", wf.get("TableWorkflowId"))
    print("Source Connection String:", source_conn_string)
    print("Target Connection String:", target_conn_string)
    print("-" * 50)
